In [4]:
import pandas as pd
import os
import chardet
import numpy as np
import glob
import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import  StandardScaler, RobustScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier


In [5]:

from plotly.subplots import make_subplots
import plotly.graph_objs as go
import pprint
from scipy import stats
from IPython.display import display

__author__ = "Juanma Hernández"
__copyright__ = "Copyright 2019"
__credits__ = ["Juanma Hernández", "George Fisher"]
__license__ = "GPL"
__maintainer__ = "Juanma Hernández"
__email__ = "https://twitter.com/juanmah"
__status__ = "Utility script"


def plot_grid_search(clf):
    """Plot as many graphs as parameters are in the grid search results.

    Each graph has the values of each parameter in the X axis and the Score in the Y axis.

    Parameters
    ----------
    clf: estimator object result of a GridSearchCV
        This object contains all the information of the cross validated results for all the parameters combinations.
    """
    # Convert the cross validated results in a DataFrame ordered by `rank_test_score` and `mean_fit_time`.
    # As it is frequent to have more than one combination with the same max score,
    # the one with the least mean fit time SHALL appear first.
    cv_results = pd.DataFrame(clf.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

    # Get parameters
    parameters=cv_results['params'][0].keys()

    # Calculate the number of rows and columns necessary
    rows = -(-len(parameters) // 2)
    columns = min(len(parameters), 2)
    # Create the subplot
    fig = make_subplots(rows=rows, cols=columns)
    # Initialize row and column indexes
    row = 1
    column = 1

    # For each of the parameters
    for parameter in parameters:

        # As all the graphs have the same traces, and by default all traces are shown in the legend,
        # the description appears multiple times. Then, only show legend of the first graph.
        if row == 1 and column == 1:
            show_legend = True
        else:
            show_legend = False

        # Mean test score
        mean_test_score = cv_results[cv_results['rank_test_score'] != 1]
        fig.add_trace(go.Scatter(
            name='Mean test score',
            x=mean_test_score['param_' + parameter],
            y=mean_test_score['mean_test_score'],
            mode='markers',
            marker=dict(size=mean_test_score['mean_fit_time'],
                        color='SteelBlue',
                        sizeref=2. * cv_results['mean_fit_time'].max() / (40. ** 2),
                        sizemin=4,
                        sizemode='area'),
            text=mean_test_score['params'].apply(
                lambda x: pprint.pformat(x, width=-1).replace('{', '').replace('}', '').replace('\n', '<br />')),
            showlegend=show_legend),
            row=row,
            col=column)

        # Best estimators
        rank_1 = cv_results[cv_results['rank_test_score'] == 1]
        fig.add_trace(go.Scatter(
            name='Best estimators',
            x=rank_1['param_' + parameter],
            y=rank_1['mean_test_score'],
            mode='markers',
            marker=dict(size=rank_1['mean_fit_time'],
                        color='Crimson',
                        sizeref=2. * cv_results['mean_fit_time'].max() / (40. ** 2),
                        sizemin=4,
                        sizemode='area'),
            text=rank_1['params'].apply(str),
            showlegend=show_legend),
            row=row,
            col=column)

        fig.update_xaxes(title_text=parameter, row=row, col=column)
        fig.update_yaxes(title_text='Score', row=row, col=column)

        # Check the linearity of the series
        # Only for numeric series
        if pd.to_numeric(cv_results['param_' + parameter], errors='coerce').notnull().all():
            x_values = cv_results['param_' + parameter].sort_values().unique().tolist()
            r = stats.linregress(x_values, range(0, len(x_values))).rvalue
            # If not so linear, then represent the data as logarithmic
            if r < 0.86:
                fig.update_xaxes(type='log', row=row, col=column)

        # Increment the row and column indexes
        column += 1
        if column > columns:
            column = 1
            row += 1

            # Show first the best estimators
    fig.update_layout(legend=dict(traceorder='reversed'),
                      width=columns * 360 + 100,
                      height=rows * 360,
                      title='Best score: {:.6f} with {}'.format(cv_results['mean_test_score'].iloc[0],
                                                                str(cv_results['params'].iloc[0]).replace('{',
                                                                                                          '').replace(
                                                                    '}', '')),
                      hovermode='closest',
                      template='none')
    fig.show()


def table_grid_search(clf, all_columns=False, all_ranks=False, save=True):
    """Show tables with the grid search results.

    Parameters
    ----------
    clf: estimator object result of a GridSearchCV
        This object contains all the information of the cross validated results for all the parameters combinations.

    all_columns: boolean, default: False
        If true all columns are returned. If false, the following columns are dropped:

        - params. As each parameter has a column with the value.
        - std_*. Standard deviations.
        - split*. Split scores.

    all_ranks: boolean, default: False
        If true all ranks are returned. If false, only the rows with rank equal to 1 are returned.

    save: boolean, default: True
        If true, results are saved to a CSV file.
    """
    # Convert the cross validated results in a DataFrame ordered by `rank_test_score` and `mean_fit_time`.
    # As it is frequent to have more than one combination with the same max score,
    # the one with the least mean fit time SHALL appear first.
    cv_results = pd.DataFrame(clf.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

    # Reorder
    columns = cv_results.columns.tolist()
    # rank_test_score first, mean_test_score second and std_test_score third
    columns = columns[-1:] + columns[-3:-1] + columns[:-3]
    cv_results = cv_results[columns]

    if save:
        cv_results.to_csv('--'.join(cv_results['params'][0].keys()) + '.csv', index=True, index_label='Id')

    # Unless all_columns are True, drop not wanted columns: params, std_* split*
    if not all_columns:
        cv_results.drop('params', axis='columns', inplace=True)
        cv_results.drop(list(cv_results.filter(regex='^std_.*')), axis='columns', inplace=True)
        cv_results.drop(list(cv_results.filter(regex='^split.*')), axis='columns', inplace=True)

    # Unless all_ranks are True, filter out those rows which have rank equal to one
    if not all_ranks:
        cv_results = cv_results[cv_results['rank_test_score'] == 1]
        cv_results.drop('rank_test_score', axis = 'columns', inplace = True)        

    display(cv_results)

In [6]:
# Define the path where your CSV files are located
csv_path = "Data/*.csv"

# Get a list of all CSV file paths matching the pattern
csv_files = glob.glob(csv_path)

# Initialize an empty dictionary to store the DataFrames
dataframes = {}
# Read each CSV file into a DataFrame and store it in the dictionary
for file in csv_files:
    # Extract the file name without the extension as the dictionary key
    file_name = file.split(".",1)[0]
    file_name = file_name.split("Data\\")[1]

        # Read the CSV file into a DataFrame
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    df = pd.read_csv(file,encoding=encoding)
        
        # Store the DataFrame in the dictionary
    dataframes[file_name] = df

# Access the individual DataFrames by their respective keys


In [7]:
maleGames = pd.DataFrame()
maleGames = pd.concat([dataframes["MRegularSeasonDetailedResults"],dataframes["MNCAATourneyDetailedResults"]], ignore_index=True)
maleGames

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108877,2022,146,1242,76,1274,50,N,0,29,58,...,21,9,13,5,21,7,14,7,4,20
108878,2022,146,1314,69,1389,49,N,0,25,61,...,16,9,10,4,25,11,7,4,7,18
108879,2022,152,1242,81,1437,65,N,0,29,54,...,31,8,10,12,17,12,9,3,0,11
108880,2022,152,1314,81,1181,77,N,0,27,64,...,22,12,20,13,25,12,4,7,4,18


In [8]:
pd.set_option('display.max_columns', 500)
maleGames
maleGames["CityID"] = 0
for x in range(34074,len(maleGames)):
    done = 0
    currGame = maleGames.iloc[x]
    hi = len(dataframes["MGameCities"])
    lo = 0
    if(currGame["Season"]>2009):
        while(not done and hi != lo):
            mid = int((hi+lo)/2)
            if(currGame["Season"]>dataframes["MGameCities"].iloc[mid]["Season"]):
                lo = mid
            elif(currGame["Season"]<dataframes["MGameCities"].iloc[mid]["Season"]):
                hi = mid
            else:
                if(currGame["DayNum"]>dataframes["MGameCities"].iloc[mid]["DayNum"]):
                    lo = mid
                elif(currGame["DayNum"]<dataframes["MGameCities"].iloc[mid]["DayNum"]):
                    hi = mid
                else:
                    if(currGame["WTeamID"]>dataframes["MGameCities"].iloc[mid]["WTeamID"]):
                        lo = mid
                    elif(currGame["WTeamID"]<dataframes["MGameCities"].iloc[mid]["WTeamID"]):
                        hi = mid
                    else:
                        maleGames.loc[x, "CityID"] = dataframes["MGameCities"].iloc[mid]["CityID"]
                        done = 1

In [9]:
maleGames = maleGames[maleGames["CityID"]!= 0]
maleGames

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CityID
34074,2010,7,1143,75,1293,70,H,0,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,4027
34075,2010,7,1314,88,1198,72,H,0,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,4061
34076,2010,7,1326,100,1108,60,H,0,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,4080
34077,2010,7,1393,75,1107,43,H,0,29,60,2,17,15,31,14,32,21,21,21,6,16,15,55,5,28,8,14,12,27,10,32,12,4,27,4340
34078,2010,9,1143,95,1178,61,H,0,29,61,7,17,30,35,15,30,11,10,5,3,20,20,57,4,18,17,27,14,18,14,15,4,2,21,4027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108877,2022,146,1242,76,1274,50,N,0,29,58,5,14,13,26,6,33,18,10,7,3,16,19,55,3,21,9,13,5,21,7,14,7,4,20,4071
108878,2022,146,1314,69,1389,49,N,0,25,61,6,22,13,21,12,33,15,6,6,5,12,18,60,4,16,9,10,4,25,11,7,4,7,18,4266
108879,2022,152,1242,81,1437,65,N,0,29,54,13,24,10,13,10,25,18,7,4,4,8,22,57,13,31,8,10,12,17,12,9,3,0,11,4237
108880,2022,152,1314,81,1181,77,N,0,27,64,10,26,17,24,15,33,7,10,2,5,19,30,72,5,22,12,20,13,25,12,4,7,4,18,4237


In [10]:
column_name_mapping = {
    'WTeamID': 'Team1',
    'WScore' : 'Team1Score',
    'LTeamID': 'Team2',
    'LScore': 'Team2Score',
    'WLoc': 'Team1Loc',
    'WFGM': 'Team1FGM',
    'WFGA': 'Team1FGA',
    'WFGM3': 'Team1FGM3',
    'WFGA3': 'Team1FGA3',
    'WFTM': 'Team1FTM',
    'WOR': 'Team1OR',
    'WDR': 'Team1DR',
    'WAst': 'Team1Ast',
    'WTO': 'Team1TO',
    'WStl': 'Team1Stl',
    'WBlk': 'Team1Blk',
    'WPF': 'Team1PF',
    'LFGM': 'Team2FGM',
    'LFGA': 'Team2FGA',
    'LFGM3': 'Team2FGM3',
    'LFGA3': 'Team2FGA3',
    'LFTM': 'Team2FTM',
    'LOR': 'Team2OR',
    'LDR': 'Team2DR',
    'LAst': 'Team2Ast',
    'LTO': 'Team2TO',
    'LStl': 'Team2Stl',
    'LBlk': 'Team2Blk',
    'LPF': 'Team2PF'
}
newDf = pd.DataFrame(np.repeat(maleGames.values, 2, axis=0))
for x in range(1,len(newDf),2):
    newDf.loc[x,2],newDf.loc[x,4] = newDf.loc[x,4],newDf.loc[x,2]
    newDf.loc[x,3],newDf.loc[x,5] = newDf.loc[x,5],newDf.loc[x,3]
    for y in range(8,21):
       newDf.loc[x,y],newDf.loc[x,y+13] = newDf.loc[x,y+13],newDf.loc[x,y]    
newDf.columns = maleGames.columns
newDf.rename(columns=column_name_mapping, inplace=True)

for x in range(len(newDf)):
    if x % 2 == 0:
        newDf.loc[x, "Win"] = 1
    newDf.loc[x, "PointDif"] = newDf.loc[x, "Team1Score"] - newDf.loc[x, "Team2Score"]
newDf

,Season,DayNum,Team1,Team1Score,Team2,Team2Score,Team1Loc,NumOT,Team1FGM,Team1FGA,Team1FGM3,Team1FGA3,Team1FTM,WFTA,Team1OR,Team1DR,Team1Ast,Team1TO,Team1Stl,Team1Blk,Team1PF,Team2FGM,Team2FGA,Team2FGM3,Team2FGA3,Team2FTM,LFTA,Team2OR,Team2DR,Team2Ast,Team2TO,Team2Stl,Team2Blk,Team2PF,CityID,Win,PointDif
0,2010,7,1143,75,1293,70,H,0,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,4027,1.0,5.0
1,2010,7,1293,70,1143,75,H,0,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,4027,NaN,-5.0
2,2010,7,1314,88,1198,72,H,0,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,4061,1.0,16.0
3,2010,7,1198,72,1314,88,H,0,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,4061,NaN,-16.0
4,2010,7,1326,100,1108,60,H,0,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,4080,1.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148715,2022,152,1437,65,1242,81,N,0,22,57,13,31,8,10,12,17,12,9,3,0,11,29,54,13,24,10,13,10,25,18,7,4,4,8,4237,NaN,-16.0
148716,2022,152,1314,81,1181,77,N,0,27,64,10,26,17,24,15,33,7,10,2,5,19,30,72,5,22,12,20,13,25,12,4,7,4,18,4237,1.0,4.0
148717,2022,152,1181,77,1314,81,N,0,30,72,5,22,12,20,13,25,12,4,7,4,18,27,64,10,26,17,24,15,33,7,10,2,5,19,4237,NaN,-4.0
148718,2022,154,1242,72,1314,69,N,0,29,66,6,17,8,14,7,27,11,9,6,5,16,23,73,5,23,18,22,20,29,9,13,2,6,13,4237,1.0,3.0


In [11]:
from scipy.stats import poisson

def prob_x_greater_than_y(lam1, lam2):
    prob = 0.0
    for k in range(int(poisson.ppf(0.99, lam1)) + 1):
        prob_k = poisson.pmf(k, lam1)
        prob_y_less_than_k = poisson.cdf(k-1, lam2)
        prob += prob_k * prob_y_less_than_k
    return 1 - prob

In [12]:

# Example usage
lambda1 = 3
lambda2 = 2
result = prob_x_greater_than_y(lambda1, lambda2)
print(f"The probability that X is greater than Y is: {result:.4f}")
        

The probability that X is greater than Y is: 0.4185


In [13]:
newDf 
newDf["Team1Seed"] = 0
newDf["Team2Seed"] = 0 
for x in range(0,len(newDf)):
    if(newDf["DayNum"].iloc[x]>131):
        currTeam1 = newDf["Team1"].iloc[x]
        currTeam2 = newDf["Team2"].iloc[x]
        currSeason = newDf["Season"].iloc[x]
        for y in range(0,len(dataframes["MNCAATourneySeeds"])):
            if(currTeam1 == dataframes["MNCAATourneySeeds"]["TeamID"].iloc[y] and currSeason == dataframes["MNCAATourneySeeds"]["Season"].iloc[y]):
                newDf.loc[x,"Team1Seed"] = dataframes["MNCAATourneySeeds"]["Seed"].iloc[y]
            if(currTeam2 == dataframes["MNCAATourneySeeds"]["TeamID"].iloc[y] and currSeason == dataframes["MNCAATourneySeeds"]["Season"].iloc[y]):
                newDf.loc[x,"Team2Seed"] = dataframes["MNCAATourneySeeds"]["Seed"].iloc[y]
copyDf = newDf.copy()

copyDf['AWin'] = (copyDf['Team1Score'] > copyDf['Team2Score']).astype(int)
copyDf['ALoss'] = (copyDf['Team1Score'] < copyDf['Team2Score']).astype(int)
copyDf['BWin'] = (copyDf['Team1Score'] < copyDf['Team2Score']).astype(int)
copyDf['BLoss'] = (copyDf['Team1Score'] > copyDf['Team2Score']).astype(int)
copyDf['AWins'] = copyDf.groupby(['Season', 'Team1'])['AWin'].transform('sum')
copyDf['ALosses'] = copyDf.groupby(['Season', 'Team1'])['ALoss'].transform('sum')
copyDf['BWins'] = copyDf.groupby(['Season', 'Team2'])['BWin'].transform('sum')
copyDf['BLosses'] = copyDf.groupby(['Season', 'Team2'])['BLoss'].transform('sum')
copyDf["TeamAWRatio"] = copyDf["AWins"] / (copyDf["AWins"] + copyDf["ALosses"])
copyDf["TeamBWRatio"] = copyDf["BWins"] / (copyDf["BWins"] + copyDf["BLosses"])
copyDf = copyDf.drop(["ALoss", "BLoss", "BWin", "AWins", "BLosses", "ALosses", "BWins"], axis= 1)
copyDf

,Season,DayNum,Team1,Team1Score,Team2,Team2Score,Team1Loc,NumOT,Team1FGM,Team1FGA,Team1FGM3,Team1FGA3,Team1FTM,WFTA,Team1OR,Team1DR,Team1Ast,Team1TO,Team1Stl,Team1Blk,Team1PF,Team2FGM,Team2FGA,Team2FGM3,Team2FGA3,Team2FTM,LFTA,Team2OR,Team2DR,Team2Ast,Team2TO,Team2Stl,Team2Blk,Team2PF,CityID,Win,PointDif,Team1Seed,Team2Seed,AWin,TeamAWRatio,TeamBWRatio
0,2010,7,1143,75,1293,70,H,0,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,4027,1.0,5.0,0,0,1,0.685714,0.852941
1,2010,7,1293,70,1143,75,H,0,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,4027,NaN,-5.0,0,0,0,0.852941,0.685714
2,2010,7,1314,88,1198,72,H,0,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,4061,1.0,16.0,0,0,1,0.500000,0.193548
3,2010,7,1198,72,1314,88,H,0,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,4061,NaN,-16.0,0,0,0,0.193548,0.500000
4,2010,7,1326,100,1108,60,H,0,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,4080,1.0,40.0,0,0,1,0.783784,0.064516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148715,2022,152,1437,65,1242,81,N,0,22,57,13,31,8,10,12,17,12,9,3,0,11,29,54,13,24,10,13,10,25,18,7,4,4,8,4237,NaN,-16.0,Z02,Y01,0,0.789474,0.850000
148716,2022,152,1314,81,1181,77,N,0,27,64,10,26,17,24,15,33,7,10,2,5,19,30,72,5,22,12,20,13,25,12,4,7,4,18,4237,1.0,4.0,W08,X02,1,0.743590,0.820513
148717,2022,152,1181,77,1314,81,N,0,30,72,5,22,12,20,13,25,12,4,7,4,18,27,64,10,26,17,24,15,33,7,10,2,5,19,4237,NaN,-4.0,X02,W08,0,0.820513,0.743590
148718,2022,154,1242,72,1314,69,N,0,29,66,6,17,8,14,7,27,11,9,6,5,16,23,73,5,23,18,22,20,29,9,13,2,6,13,4237,1.0,3.0,Y01,W08,1,0.850000,0.743590


In [14]:

T1 = copyDf.copy()
T2 = copyDf.copy()
T1.columns = ["Team1" + x.replace("Team1","").replace("Team2","opponent_") for x in list(T1.columns)]
T2.columns = ["Team2" + x.replace("Team1","").replace("Team2","opponent_") for x in list(T2.columns)]
features = ['Team1FGM', 'Team1FGA', 'Team1FGM3', 'Team1FGA3', 'Team1FTM', 'WFTA' , 'Team1OR', 'Team1DR', 'Team1Ast', 'Team1TO', 'Team1Stl' , 'Team1Blk', 'Team1PF', 'Team2FGM', 'Team2FGA', 'Team2FGM3', 'Team2FGA3',  'Team2FTM', 'LFTA',
                                                 'Team2OR', 'Team2DR', 'Team2Ast', 'Team2TO', 'Team2Stl', 'Team2Blk', 'Team2PF', 'PointDif']
meanDf = copyDf.groupby(["Season", 'Team1'])[features].mean().reset_index()

T2.columns.values[0] = "Season"
T1.columns.values[0] = "Season"
T1.columns.values[1] = "DayNum"
T2.columns.values[1] = "DayNum"
T2 = T2.drop(['Team2Score', 'Team2opponent_', 'Team2opponent_Score', 'Team2Loc', 'Team2NumOT', 'Team2opponent_Seed', 'Team2CityID','Team2Seed', 'Team2Win', 'Team2TeamAWRatio','Team2TeamBWRatio' , 'Team2AWin'], axis= 1)

T1 = T1.drop(['Team1Score', 'Team1opponent_', 'Team1opponent_Score', 'Team1Loc', 'Team1NumOT', 'Team1opponent_Seed', 'Team1CityID','Team1Seed', 'Team1Win', 'Team1TeamAWRatio','Team1TeamBWRatio', 'Team1AWin' ], axis= 1)
copyDf


,Season,DayNum,Team1,Team1Score,Team2,Team2Score,Team1Loc,NumOT,Team1FGM,Team1FGA,Team1FGM3,Team1FGA3,Team1FTM,WFTA,Team1OR,Team1DR,Team1Ast,Team1TO,Team1Stl,Team1Blk,Team1PF,Team2FGM,Team2FGA,Team2FGM3,Team2FGA3,Team2FTM,LFTA,Team2OR,Team2DR,Team2Ast,Team2TO,Team2Stl,Team2Blk,Team2PF,CityID,Win,PointDif,Team1Seed,Team2Seed,AWin,TeamAWRatio,TeamBWRatio
0,2010,7,1143,75,1293,70,H,0,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,4027,1.0,5.0,0,0,1,0.685714,0.852941
1,2010,7,1293,70,1143,75,H,0,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,4027,NaN,-5.0,0,0,0,0.852941,0.685714
2,2010,7,1314,88,1198,72,H,0,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,4061,1.0,16.0,0,0,1,0.500000,0.193548
3,2010,7,1198,72,1314,88,H,0,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,4061,NaN,-16.0,0,0,0,0.193548,0.500000
4,2010,7,1326,100,1108,60,H,0,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,4080,1.0,40.0,0,0,1,0.783784,0.064516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148715,2022,152,1437,65,1242,81,N,0,22,57,13,31,8,10,12,17,12,9,3,0,11,29,54,13,24,10,13,10,25,18,7,4,4,8,4237,NaN,-16.0,Z02,Y01,0,0.789474,0.850000
148716,2022,152,1314,81,1181,77,N,0,27,64,10,26,17,24,15,33,7,10,2,5,19,30,72,5,22,12,20,13,25,12,4,7,4,18,4237,1.0,4.0,W08,X02,1,0.743590,0.820513
148717,2022,152,1181,77,1314,81,N,0,30,72,5,22,12,20,13,25,12,4,7,4,18,27,64,10,26,17,24,15,33,7,10,2,5,19,4237,NaN,-4.0,X02,W08,0,0.820513,0.743590
148718,2022,154,1242,72,1314,69,N,0,29,66,6,17,8,14,7,27,11,9,6,5,16,23,73,5,23,18,22,20,29,9,13,2,6,13,4237,1.0,3.0,Y01,W08,1,0.850000,0.743590


In [15]:
T2.head()

,Season,DayNum,Team2,Team2FGM,Team2FGA,Team2FGM3,Team2FGA3,Team2FTM,Team2WFTA,Team2OR,Team2DR,Team2Ast,Team2TO,Team2Stl,Team2Blk,Team2PF,Team2opponent_FGM,Team2opponent_FGA,Team2opponent_FGM3,Team2opponent_FGA3,Team2opponent_FTM,Team2LFTA,Team2opponent_OR,Team2opponent_DR,Team2opponent_Ast,Team2opponent_TO,Team2opponent_Stl,Team2opponent_Blk,Team2opponent_PF,Team2PointDif
0,2010,7,1143,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,5.0
1,2010,7,1293,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,-5.0
2,2010,7,1314,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,16.0
3,2010,7,1198,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,-16.0
4,2010,7,1326,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,40.0


In [16]:
T1.head()

,Season,DayNum,Team1,Team1FGM,Team1FGA,Team1FGM3,Team1FGA3,Team1FTM,Team1WFTA,Team1OR,Team1DR,Team1Ast,Team1TO,Team1Stl,Team1Blk,Team1PF,Team1opponent_FGM,Team1opponent_FGA,Team1opponent_FGM3,Team1opponent_FGA3,Team1opponent_FTM,Team1LFTA,Team1opponent_OR,Team1opponent_DR,Team1opponent_Ast,Team1opponent_TO,Team1opponent_Stl,Team1opponent_Blk,Team1opponent_PF,Team1PointDif
0,2010,7,1143,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,5.0
1,2010,7,1293,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,-5.0
2,2010,7,1314,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,16.0
3,2010,7,1198,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,-16.0
4,2010,7,1326,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,40.0


In [17]:
regSeason = copyDf[['Season', 'DayNum', 'Team1', 'Team1Score', 'Team2', 'Team2Score', 'TeamAWRatio', 'TeamBWRatio']]

reg = regSeason.merge(T1, on = ['Season', 'Team1', 'DayNum' ])

reg = reg.merge(T2, on = ['Season', 'Team2', 'DayNum'])
reg['AWin'] = (reg['Team1Score'] > reg['Team2Score']).astype(int)
reg = reg.drop(["Team1PointDif", 'Team2PointDif'], axis= 1)

reg

,Season,DayNum,Team1,Team1Score,Team2,Team2Score,TeamAWRatio,TeamBWRatio,Team1FGM,Team1FGA,Team1FGM3,Team1FGA3,Team1FTM,Team1WFTA,Team1OR,Team1DR,Team1Ast,Team1TO,Team1Stl,Team1Blk,Team1PF,Team1opponent_FGM,Team1opponent_FGA,Team1opponent_FGM3,Team1opponent_FGA3,Team1opponent_FTM,Team1LFTA,Team1opponent_OR,Team1opponent_DR,Team1opponent_Ast,Team1opponent_TO,Team1opponent_Stl,Team1opponent_Blk,Team1opponent_PF,Team2FGM,Team2FGA,Team2FGM3,Team2FGA3,Team2FTM,Team2WFTA,Team2OR,Team2DR,Team2Ast,Team2TO,Team2Stl,Team2Blk,Team2PF,Team2opponent_FGM,Team2opponent_FGA,Team2opponent_FGM3,Team2opponent_FGA3,Team2opponent_FTM,Team2LFTA,Team2opponent_OR,Team2opponent_DR,Team2opponent_Ast,Team2opponent_TO,Team2opponent_Stl,Team2opponent_Blk,Team2opponent_PF,AWin
0,2010,7,1143,75,1293,70,0.685714,0.852941,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,1
1,2010,7,1293,70,1143,75,0.852941,0.685714,26,52,8,21,10,15,11,20,11,17,7,3,24,24,52,5,12,22,32,13,19,15,12,8,1,14,24,52,5,12,22,32,13,19,15,12,8,1,14,26,52,8,21,10,15,11,20,11,17,7,3,24,0
2,2010,7,1314,88,1198,72,0.500000,0.193548,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,1
3,2010,7,1198,72,1314,88,0.193548,0.500000,25,68,8,23,14,17,13,16,15,20,14,2,18,34,61,4,13,16,19,12,32,23,26,10,8,19,34,61,4,13,16,19,12,32,23,26,10,8,19,25,68,8,23,14,17,13,16,15,20,14,2,18,0
4,2010,7,1326,100,1108,60,0.783784,0.064516,39,73,14,33,8,12,13,34,29,6,8,6,19,21,61,7,17,11,20,11,24,7,13,4,5,16,21,61,7,17,11,20,11,24,7,13,4,5,16,39,73,14,33,8,12,13,34,29,6,8,6,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148715,2022,152,1437,65,1242,81,0.789474,0.850000,22,57,13,31,8,10,12,17,12,9,3,0,11,29,54,13,24,10,13,10,25,18,7,4,4,8,29,54,13,24,10,13,10,25,18,7,4,4,8,22,57,13,31,8,10,12,17,12,9,3,0,11,0
148716,2022,152,1314,81,1181,77,0.743590,0.820513,27,64,10,26,17,24,15,33,7,10,2,5,19,30,72,5,22,12,20,13,25,12,4,7,4,18,30,72,5,22,12,20,13,25,12,4,7,4,18,27,64,10,26,17,24,15,33,7,10,2,5,19,1
148717,2022,152,1181,77,1314,81,0.820513,0.743590,30,72,5,22,12,20,13,25,12,4,7,4,18,27,64,10,26,17,24,15,33,7,10,2,5,19,27,64,10,26,17,24,15,33,7,10,2,5,19,30,72,5,22,12,20,13,25,12,4,7,4,18,0
148718,2022,154,1242,72,1314,69,0.850000,0.743590,29,66,6,17,8,14,7,27,11,9,6,5,16,23,73,5,23,18,22,20,29,9,13,2,6,13,23,73,5,23,18,22,20,29,9,13,2,6,13,29,66,6,17,8,14,7,27,11,9,6,5,16,1


In [56]:
y = reg["AWin"]
x = reg.iloc[:, np.r_[0,2,4, 6:60]]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 0)

pred_Y = {"randomforest" : {}, "LogRegression": {}, "XGBoost": {}, "Bagging": {}, "SVM" : {} }



In [19]:
x_copy = x.copy()
threshold = [0.05, 0.01, 0.005, 0.001]
num_features = {}
for i in range (0, 4):
    x_copy = x.copy()
    feature_selection = VarianceThreshold(threshold = threshold[i] )
    feature_selection.fit_transform(x_copy)
    num_features[threshold[i]] = x.shape[1]
num_features

{0.05: 57, 0.01: 57, 0.005: 57, 0.001: 57}

In [20]:
scalers = [
    StandardScaler(),
    RobustScaler(),
    MinMaxScaler(),
]


k_fold = KFold(n_splits=5, shuffle=True, random_state=0)


for i in range(0, 3):
    X_traincopy = X_train.copy()
    scalers[i].fit_transform(X_traincopy)
    RF = RandomForestClassifier(n_estimators= 100, max_depth = None)
    RF.fit(X_train, y_train)
    pred_Y["randomforest"] = RF.predict_proba(X_test)
    print(brier_score_loss(y_test, pred_Y["randomforest"][:,1]
    ))  


 

0.057165034965034965
0.055567592343553886
0.05646255379236148


In [57]:
RF2 = RandomForestClassifier(n_estimators= 100, max_depth= None)
scaler = StandardScaler()
scaler.fit(X_train)
RF2.fit(X_train, y_train)
sel = SelectFromModel(RF2)
sel.get_support()
X_filter_test = sel.transform(X_test)
X_filter_train = sel.transform(X_train)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:457: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:457: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names



In [22]:

arr = pd.DataFrame( RF2.feature_importances_,RF2.feature_names_in_, columns=['Importance'])
arr = arr.sort_values(by=['Importance'], ascending= False)
arr

,Importance
Team1opponent_FGM,0.051821
Team2opponent_FGM,0.051303
Team2FGM,0.050063
Team1FGM,0.046512
Team1DR,0.044105
Team1opponent_DR,0.044085
Team2opponent_DR,0.041575
Team2DR,0.036973
Team1Ast,0.032442
TeamBWRatio,0.032373


In [70]:
scores = {}
detailed_results= {}

In [119]:
params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 6],
}

grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=params, scoring= 'neg_brier_score', cv = 5)
grid_result = grid_search.fit(X_test, y_test)
scores['randomforest_train'] = grid_search.best_score_

scores['randomforest_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_test)[:,1]
 )




In [122]:
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])
detailed_results["RandomForest"] = cv_results.iloc[0]
plot_grid_search(grid_result)



In [121]:
grid_result = grid_search.fit(X_filter_test, y_test)
print(grid_search.best_score_)
scores['randomforest_filter_train'] = grid_result.best_score_

grid_search.best_estimator_.predict_proba(X_filter_test)
scores['randomforest_filter_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_filter_test)[:,1]
 )
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

detailed_results["RandomForest_filter"] = cv_results.iloc[0]



-0.1050740145485646


In [75]:
learning_rate = [ 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
max_depth = [3, 4, 5, 6]


param_grid = dict(learning_rate=learning_rate, max_depth = max_depth,  gamma = [0, 0.5, 1, 1.5])
XG = XGBClassifier()

X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)


grid_search = GridSearchCV(XG, param_grid, scoring="neg_brier_score", cv=k_fold)

grid_result = grid_search.fit(X_test, y_test)



In [76]:
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])
detailed_results["XG"] = cv_results.iloc[0]


display(cv_results)
scores['xg_train'] = grid_result.best_score_
plot_grid_search(grid_result)
scores['xg_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_test)[:,1]
 )

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_learning_rate,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
93,0.999660,0.007881,0.005386,0.000489,1.5,0.8,4,"{'gamma': 1.5, 'learning_rate': 0.8, 'max_dept...",-0.011448,-0.011964,-0.011866,-0.012719,-0.012700,-0.012139,0.000497,1
21,1.027051,0.029779,0.005585,0.000489,0,0.8,4,"{'gamma': 0, 'learning_rate': 0.8, 'max_depth'...",-0.010522,-0.012007,-0.014144,-0.012574,-0.012493,-0.012348,0.001162,2
45,1.008138,0.009028,0.005585,0.000489,0.5,0.8,4,"{'gamma': 0.5, 'learning_rate': 0.8, 'max_dept...",-0.010147,-0.012149,-0.014066,-0.012914,-0.012975,-0.012450,0.001304,3
69,1.009017,0.009224,0.005186,0.000399,1,0.8,4,"{'gamma': 1, 'learning_rate': 0.8, 'max_depth'...",-0.011524,-0.013018,-0.012503,-0.012980,-0.013284,-0.012662,0.000622,4
42,1.245580,0.008996,0.005785,0.000746,0.5,0.7,5,"{'gamma': 0.5, 'learning_rate': 0.7, 'max_dept...",-0.012566,-0.014320,-0.013220,-0.012044,-0.012820,-0.012994,0.000765,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.778745,0.004535,0.005186,0.000746,1.5,0.4,3,"{'gamma': 1.5, 'learning_rate': 0.4, 'max_dept...",-0.018467,-0.018939,-0.019464,-0.018784,-0.019561,-0.019043,0.000414,92
72,0.782435,0.013116,0.005186,0.000746,1.5,0.3,3,"{'gamma': 1.5, 'learning_rate': 0.3, 'max_dept...",-0.023345,-0.023226,-0.023449,-0.022587,-0.023072,-0.023136,0.000302,93
48,0.783434,0.006211,0.005585,0.000797,1,0.3,3,"{'gamma': 1, 'learning_rate': 0.3, 'max_depth'...",-0.023345,-0.023066,-0.023627,-0.022821,-0.023076,-0.023187,0.000276,94
0,0.793398,0.023442,0.004787,0.000399,0,0.3,3,"{'gamma': 0, 'learning_rate': 0.3, 'max_depth'...",-0.023345,-0.023406,-0.023629,-0.022821,-0.022966,-0.023233,0.000297,95


In [77]:
grid_result = grid_search.fit(X_filter_test, y_test)
scores['xg_filter'] = grid_result.best_score_
plot_grid_search(grid_result)
scores['xg_filter_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_filter_test)[:,1]
 )
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

detailed_results["XG_filter"] = cv_results.iloc[0]



In [93]:
sgd = SGDClassifier(loss = 'log_loss')
parameters = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2], # learning rate
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'penalty':  ['l1', 'l2', 'elasticnet'],
    'eta0': [1, 10, 100], 

}
grid_search = GridSearchCV(sgd, parameters, cv=k_fold,  scoring='neg_brier_score')
grid_result = grid_search.fit(X_test, y_test)


In [94]:
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])
detailed_results["SGD"] = cv_results.iloc[0]
display(cv_results)
scores['sgd_train'] = grid_result.best_score_
plot_grid_search(grid_result)
scores['sgd_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_test)[:,1]
 )

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_eta0,param_learning_rate,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.193898,0.009291,0.003785,0.000752,0.0001,1,constant,l1,"{'alpha': 0.0001, 'eta0': 1, 'learning_rate': ...",-5.276398e-06,-1.223817e-05,-0.000006,-0.000011,-4.119584e-06,-0.000008,0.000003,1
6,1.246230,0.011384,0.003599,0.000485,0.0001,1,adaptive,l1,"{'alpha': 0.0001, 'eta0': 1, 'learning_rate': ...",-2.912014e-05,-4.142365e-05,-0.000036,-0.000021,-1.396813e-05,-0.000028,0.000010,2
15,1.689310,0.026764,0.003181,0.000404,0.0001,10,adaptive,l1,"{'alpha': 0.0001, 'eta0': 10, 'learning_rate':...",-9.727982e-05,-6.424779e-08,-0.000089,-0.000083,-3.982119e-05,-0.000062,0.000037,3
12,0.190504,0.005023,0.004194,0.000396,0.0001,10,invscaling,l1,"{'alpha': 0.0001, 'eta0': 10, 'learning_rate':...",-5.647995e-05,-4.392115e-05,-0.000033,-0.000107,-7.972870e-05,-0.000064,0.000026,4
24,1.171013,0.062109,0.003994,0.001258,0.0001,100,adaptive,l1,"{'alpha': 0.0001, 'eta0': 100, 'learning_rate'...",-3.717445e-13,-2.588793e-23,-0.000112,-0.000211,-2.690887e-25,-0.000065,0.000085,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,0.126573,0.023158,0.003187,0.000403,0.1,100,constant,l2,"{'alpha': 0.1, 'eta0': 100, 'learning_rate': '...",-4.871134e-01,-5.011767e-01,-0.498755,-0.506892,-4.998319e-01,-0.498754,0.006462,185
155,0.282322,0.047514,0.003175,0.000975,10.0,100,constant,elasticnet,"{'alpha': 10.0, 'eta0': 100, 'learning_rate': ...",-5.128866e-01,-4.988233e-01,-0.498823,-0.493108,-4.998319e-01,-0.500695,0.006541,186
126,0.259991,0.068709,0.003585,0.000483,1.0,100,constant,l1,"{'alpha': 1.0, 'eta0': 100, 'learning_rate': '...",-5.128866e-01,-5.011767e-01,-0.501177,-0.493108,-4.998319e-01,-0.501636,0.006372,187
182,0.256521,0.098912,0.003981,0.000637,100.0,100,constant,elasticnet,"{'alpha': 100.0, 'eta0': 100, 'learning_rate':...",-5.128866e-01,-4.988233e-01,-0.498823,-0.506892,-5.001681e-01,-0.503519,0.005558,188


In [ ]:
grid_result = grid_search.fit(X_filter_test, y_test)
scores['sgd_filter_test'] = grid_result.best_score_
plot_grid_search(grid_result)
scores['sgd_filter_train']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_filter_test)[:,1]
 )
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

detailed_results["SGD_filter"] = cv_results.iloc[0]


In [105]:

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
parameters = {
    'var_smoothing' : [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
}
grid_search = GridSearchCV(gnb, parameters, cv=k_fold,  scoring='neg_brier_score')
grid_result = grid_search.fit(X_filter_test, y_test)
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])
display(cv_results)
scores['gnb_train'] = grid_result.best_score_
scores['gnb_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_filter_test)[:,1]
)
plot_grid_search(grid_result)
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

detailed_results["gnb"] = cv_results.iloc[0]


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.043283,0.003531,0.012168,0.001918,0.0,{'var_smoothing': 1e-09},-0.073057,-0.074757,-0.073861,-0.076898,-0.076508,-0.075016,0.001484,1
1,0.042593,0.001463,0.010384,0.000788,0.0,{'var_smoothing': 1e-08},-0.073057,-0.074757,-0.073861,-0.076898,-0.076508,-0.075016,0.001484,2
2,0.040302,0.001843,0.010372,0.001183,0.0,{'var_smoothing': 1e-07},-0.073057,-0.074757,-0.073861,-0.076899,-0.076508,-0.075016,0.001484,3
3,0.051351,0.007941,0.013348,0.002166,0.000001,{'var_smoothing': 1e-06},-0.073062,-0.074762,-0.073865,-0.076903,-0.076514,-0.075021,0.001484,4
4,0.043613,0.011842,0.011370,0.000791,0.00001,{'var_smoothing': 1e-05},-0.073114,-0.074811,-0.073906,-0.076948,-0.076570,-0.075070,0.001485,5
5,0.043095,0.007628,0.009977,0.000631,0.0001,{'var_smoothing': 0.0001},-0.073682,-0.075340,-0.074343,-0.077423,-0.077151,-0.075588,0.001487,6
8,0.040889,0.002755,0.010752,0.001309,0.1,{'var_smoothing': 0.1},-0.077733,-0.078578,-0.077711,-0.079742,-0.080425,-0.078838,0.001086,7
6,0.041289,0.004368,0.011763,0.001155,0.001,{'var_smoothing': 0.001},-0.077339,-0.078710,-0.077241,-0.080502,-0.080541,-0.078867,0.001447,8
7,0.041910,0.003575,0.011166,0.000746,0.01,{'var_smoothing': 0.01},-0.080115,-0.081267,-0.079606,-0.082833,-0.083021,-0.081368,0.001383,9
9,0.046368,0.003123,0.014365,0.005141,1,{'var_smoothing': 1},-0.093008,-0.092703,-0.093113,-0.093309,-0.093778,-0.093182,0.000357,10


In [102]:
grid_result = grid_search.fit(X_filter_test, y_test)
scores['gnb_filter_train'] = grid_result.best_score_
scores['gnb_filter_test']  = brier_score_loss(y_test,grid_search.best_estimator_.predict_proba(X_filter_test)[:,1]
)
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score', 'mean_fit_time'])

detailed_results["gnb_filter"] = cv_results.iloc[0]
plot_grid_search(grid_result)


In [106]:

scores


{'randomforest_train': -0.1058441180096595,
 'randomforest_test': 0.10264147424259526,
 'randomforest_filter_train': -0.10548705178653828,
 'randomforest_filter_test': 0.10195594582365541,
 'xg_train': -0.012139449138344699,
 'xg_test': 0.002293274106850247,
 'xg_filter': -0.0423634625791115,
 'xg_filter_test': 0.035641055691622306,
 'sgd_train': -7.767831635812399e-06,
 'sgd_test': 7.1648354822893254e-06,
 'sgd_filter_test': -0.04493039638042784,
 'sgd_filter_train': 0.0437908545533549,
 'gnb': -0.0750159663611173,
 'gnb_filter_train': -0.0750159663611173,
 'gnb_filter_test': 0.07497361939904136,
 'gnb_train': -0.0750159663611173,
 'gnb_test': 0.07497361939904136}

In [118]:
detailed_results['RandomForest']

mean_fit_time                                      3.317147
std_fit_time                                       0.049738
mean_score_time                                    0.084717
std_score_time                                     0.002149
param_max_depth                                           6
param_n_estimators                                      100
params                {'max_depth': 6, 'n_estimators': 100}
split0_test_score                                 -0.106983
split1_test_score                                 -0.103451
split2_test_score                                 -0.106696
split3_test_score                                 -0.104693
split4_test_score                                 -0.107397
mean_test_score                                   -0.105844
std_test_score                                     0.001516
rank_test_score                                           1
Name: 4, dtype: object

In [85]:
# Create a dictionary to store teams and their games
teams = {}

# Loop through maleGames DataFrame
for x in range(len(newDf)):
    team1 = newDf.iloc[x]["Team1"]
    
    # Add winning team to the dictionary if not already present
    if team1 not in teams:
        teams[team1] = pd.DataFrame(columns=newDf.columns)
    
    # Append the game to the respective team's DataFrame
    teams[team1] = pd.concat([teams[team1], newDf.iloc[[x]]], ignore_index=True)


KeyboardInterrupt: 

In [ ]:
# Loop through all teams
for team_id, games in teams.items():
    columns = len(teams[team_id].columns)
    for variable in range(0,columns):
        column_name = teams[team_id].columns.tolist()[variable]
        if isinstance(teams[team_id].iloc[2][variable],(int,float)):
            total = 0
            attempts = 0
            # 'games' is a list of games for the current team, you can perform operations on them here
            for x in range(1,len(teams[team_id])):
                total = total + teams[team_id].iloc[x][variable]
                attempts = attempts + 1
            teams[team_id]["avg_"+column_name] = (total/attempts)


TypeError: unsupported operand type(s) for +: 'int' and 'str'